In [ ]:
from sentence_transformers import SentenceTransformer 
import numpy as np
import json
import gradio as gr

modele = SentenceTransformer('all-mpnet-base-v2') #Chargement d'un modèle SBERT

#def Sauvegarde_JSON (Fichier):
    #"""Sauvegarde les embeddings sous format JSON """
    #with open ('Projets/Mini/data.json','w') as json_file:
        #json.dump(Charger_Documents(Fichier),json_file,indent= 4) # stocke les embeddings des documents dans un fichier JSON

def Charger_Documents(Fichier):
    """Charge les documents/Articles du Fichier et renvoie un dictrionnaire de format {Article (article) : Embeddings (valeur)"""
    global Documents_Embeddings
    Documents_Embeddings = [] 
    
    #Intègre chaque document dans une liste
    liste_Documents = []
    with open(Fichier.name, 'r') as Fichier:
        for Document in Fichier:
            liste_Documents.append(Document.strip())

    #Créer les embeddings pour tous les documents en une seule fois
    embeddings = modele.encode(liste_Documents)
    for i, (Document, embedding) in enumerate(zip(liste_Documents, embeddings), start=1):
        #Création d'un tuple (i, (articles, embeddigs)) et le parcours pour créer un dictionnaire pour chaque article
        Documents_Embeddings.append({
        "id": i,
        "texte": Document,
        "embedding": embedding.tolist() 
    })
    
    #"Sauvegarde les embeddings sous format JSON
    with open ('Projets/Mini/data.json','w') as json_file:
        json.dump(Documents_Embeddings,json_file,indent= 4) # stocke les embeddings des documents dans un fichier JSON

    return "Documents chargés"

#Charger_Documents('/Users/lnberroug/Documents/LDDBI-L2/Info/IN304/Projet/Projets/Mini/lois.txt')
#Charger_Documents('Projets/Mini/lois.txt')


def Encoder_Requete(requete):
    """Encode la requête saisie par l'utilisateur"""
    embedding_requete = modele.encode(requete) # convertie la requête en un vecteur 
    return embedding_requete


def cosine_similarity(vector1, vector2):
    """Calcul de la similarité cosinus""" # source : https://medium.com/@santannalouis208/la-similarité-cosinus-en-ia-nlp-d554d3b14efa
    #Produit scalaire des vecteurs
    scalar_product = np.dot(vector1, vector2)
    #Norme euclidienne des vecteurs
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    #Expression analytique du cosinus dans un espace euclidien
    cosine = scalar_product / (norm_vector1 * norm_vector2)
    return cosine

def Similarite_Requete_Document(requete_utilisateur):
    """Calculez la similarité cosinus entre le vecteur de la requête et les vecteurs des documents pour identifier les documents les plus proches"""
    if not Documents_Embeddings:
        return "Aucun document chargé. Veuillez d'abord charger un fichier."
    
    vecteur_requete = Encoder_Requete(requete_utilisateur)
    similarities = [
        {
            'texte' : document['texte'],
            'score' : cosine_similarity(document['embedding'], vecteur_requete)
        }
        for document in Documents_Embeddings
    ]

    score_trie = sorted(similarities, key = lambda x: x['score'], reverse = True)
    Top_3 = score_trie[:3]

    result = "\n".join([f"{doc['texte']}\nScore de similarité: {doc['score']}" for i, doc in enumerate(Top_3, 1)])
    return result


#INTERFACE GRADIO

def Charger_Fichier_Interface(Fichier):
    result = Charger_Documents(Fichier)  
    return result

def Recherche_Similarite_Interface(requete):
    result = Similarite_Requete_Document(requete)  
    return result

#l'onglet  pour charger des documents 
interface_charger = gr.Interface(
    fn=Charger_Fichier_Interface,
    inputs=gr.File(label="Charger le fichier texte"),
    outputs="text",
    title="Charger et Sauvegarder des Documents",
    description="Cliquez sur le bouton pour charger et sauvegarder les documents avec leurs embeddings sous forme de fichier JSON."
)

#l'onglet pour la recherche
interface_recherche = gr.Interface(
    fn=Recherche_Similarite_Interface,
    inputs=gr.Textbox(label="Entrez votre requête", placeholder="Saisir une requête ici...", lines=2),
    outputs="text",
    title="Recherche d'Articles",
    description="Entrez votre requête et obtenez les articles les plus pertinents."
)

#lancer l'interface
demo = gr.TabbedInterface([interface_recherche, interface_charger], ["Recherche", "Charger Document"])
demo.launch()




In [ ]:
from sentence_transformers import SentenceTransformer 
import numpy as np
import json
import gradio as gr

modele = SentenceTransformer('all-mpnet-base-v2') #Chargement d'un modèle SBERT

def Sauvegarde_JSON (Fichier):
    """Sauvegarde les embeddings sous format JSON """
    with open ('/Users/lnberroug/Documents/LDDBI-L2/Info/IN304/Projet/Projets.json','w') as json_file:
        json.dump(Fichier,json_file,indent= 4) # stocke les embeddings des documents dans un fichier JSON
    return Fichier

def Charger_Documents(Fichier):
    """Charge les documents/Articles du Fichier et renvoie un dictrionnaire de format {Article (article) : Embeddings (valeur)"""
    global Documents_Embeddings
    Documents_Embeddings = [] 
    
    #Intègre chaque document dans une liste
    liste_Documents = []
    with open(Fichier.name, 'r') as Fichier:
        for Document in Fichier:
            liste_Documents.append(Document.strip())

    #Créer les embeddings pour tous les documents en une seule fois
    embeddings = modele.encode(liste_Documents)
    for i, (Document, embedding) in enumerate(zip(liste_Documents, embeddings), start=1):
        #Création d'un tuple (i, (articles, embeddigs)) et le parcours pour créer un dictionnaire pour chaque article
        Documents_Embeddings.append({
        "id": i,
        "texte": Document,
        "embedding": embedding.tolist() 
    })
        
    Sauvegarde_JSON (Documents_Embeddings)
    return "Documents chargés"

#Charger_Documents('/Users/lnberroug/Documents/LDDBI-L2/Info/IN304/Projet/Projets/Mini/lois.txt')
#Charger_Documents('Projets/Mini/lois.txt')


def Encoder_Requete(requete):
    """Encode la requête saisie par l'utilisateur"""
    embedding_requete = modele.encode(requete) # convertie la requête en un vecteur 
    return embedding_requete



def Similarite_Requete_Document(requete_utilisateur):
    """Calculez la similarité cosinus entre le vecteur de la requête et les vecteurs des documents pour identifier les documents les plus proches"""
    if not Documents_Embeddings:
        return "Aucun document chargé. Veuillez d'abord charger un fichier."
    
    vecteur_requete = Encoder_Requete(requete_utilisateur)
    

    # convertir tous les embeddings des documents en une matrice numpy : permet de calculer la similarité cosinus en une seule opération

    embeddings_documents = np.array([doc['embedding'] for doc in Documents_Embeddings]) 
    
    #normalisation des vecteurs
    
    norm_embeddings_documents = embeddings_documents / np.linalg.norm(embeddings_documents)
    norm_vecteur_requete = vecteur_requete/np.linalg.norm(vecteur_requete)
    
    similarites = np.dot(norm_embeddings_documents,norm_vecteur_requete)
    

    similarities = [
        {
            'texte' : document['texte'],
            'score' : similarites[i]
        }
        for i, document in enumerate(Documents_Embeddings) 

        ]
    
    score_trie = sorted(similarities, key = lambda x: x['score'], reverse = True)
    Top_3 = score_trie[:3]

    result = "\n".join([f"{doc['texte']}\nScore de similarité: {doc['score']}" for i, doc in enumerate(Top_3, 1)])
    return result


#INTERFACE GRADIO

def Charger_Fichier_Interface(Fichier):
    result = Charger_Documents(Fichier)  
    return result

def Recherche_Similarite_Interface(requete):
    result = Similarite_Requete_Document(requete)  
    return result

#l'onglet  pour charger des documents 
interface_charger = gr.Interface(
    fn=Charger_Fichier_Interface,
    inputs=gr.File(label="Charger le fichier texte"),
    outputs="text",
    title="Charger et Sauvegarder des Documents",
    description="Cliquez sur le bouton pour charger et sauvegarder les documents avec leurs embeddings sous forme de fichier JSON."
)

#l'onglet pour la recherche 
interface_recherche = gr.Interface(
    fn=Recherche_Similarite_Interface,
    inputs=gr.Textbox(label="Entrez votre requête", placeholder="Saisir une requête ici...", lines=2),
    outputs="text",
    title="Recherche d'Articles",
    description="Entrez votre requête et obtenez les articles les plus pertinents."
)

#lancer l'interface
demo = gr.TabbedInterface([interface_recherche, interface_charger], ["Recherche", "Charger Document"])
demo.launch()

# hello

In [6]:
from sentence_transformers import SentenceTransformer 
import numpy as np
import json
import gradio as gr
import tempfile
import os

modele = SentenceTransformer('all-mpnet-base-v2') #Chargement d'un modèle SBERT

def Sauvegarde_JSON (Fichier):
    """Sauvegarde les embeddings sous format JSON """
    global chemin_json  
    repertoire_temporaire = tempfile.mkdtemp()
    chemin_json = os.path.join(repertoire_temporaire, 'data.json') 


    with open (chemin_json,'w') as json_file:
        json.dump(Fichier,json_file,indent= 4) # stocke les embeddings des documents dans un fichier JSON
    return chemin_json

def Charger_Documents(Fichier):
    """Charge les documents/Articles du Fichier et renvoie un dictrionnaire de format {Article (article) : Embeddings (valeur)"""
    global Documents_Embeddings
    Documents_Embeddings = [] 
    
    #Intègre chaque document dans une liste
    liste_Documents = []
    with open(Fichier.name, 'r') as Fichier:
        for Document in Fichier:
            liste_Documents.append(Document.strip())

    #Créer les embeddings pour tous les documents en une seule fois
    embeddings = modele.encode(liste_Documents)
    for i, (Document, embedding) in enumerate(zip(liste_Documents, embeddings), start=1):
        #Création d'un tuple (i, (articles, embeddigs)) et le parcours pour créer un dictionnaire pour chaque article
        Documents_Embeddings.append({
        "id": i,
        "texte": Document,
        "embedding": embedding.tolist() 
    })
    chemin_fichier_json = Sauvegarde_JSON(Documents_Embeddings)

    return  chemin_fichier_json



def Encoder_Requete(requete):
    """Encode la requête saisie par l'utilisateur"""
    embedding_requete = modele.encode(requete) # convertie la requête en un vecteur 
    return embedding_requete



def Similarite_Requete_Document(requete_utilisateur):
    """Calculez la similarité cosinus entre le vecteur de la requête et les vecteurs des documents pour identifier les documents les plus proches"""
    if not Documents_Embeddings:
        return "Aucun document chargé. Chargez un fichier."
    
    vecteur_requete = Encoder_Requete(requete_utilisateur)

    # convertir tous les embeddings des documents en une matrice numpy : permet de calculer la similarité cosinus en une seule opération

    embeddings_documents = np.array([doc['embedding'] for doc in Documents_Embeddings]) 
    
    #normalisation des vecteurs
    
    norm_embeddings_documents = embeddings_documents / np.linalg.norm(embeddings_documents)
    norm_vecteur_requete = vecteur_requete/np.linalg.norm(vecteur_requete)
    
    similarites = np.dot(norm_embeddings_documents,norm_vecteur_requete)
    

    similarities = [
        {
            'texte' : document['texte'],
            'score' : similarites[i]
        }
        for i, document in enumerate(Documents_Embeddings) 

        ]
    
    score_trie = sorted(similarities, key = lambda x: x['score'], reverse = True)
    Top_3 = score_trie[:3]

    result = "\n".join([f"{doc['texte']}\nScore de similarité: {doc['score']}" for i, doc in enumerate(Top_3, 1)])
    return result


#INTERFACE GRADIO

def Charger_Fichier_Interface(Fichier):
    chemin_fichier_json = Charger_Documents(Fichier)  
    return chemin_fichier_json

def Recherche_Similarite_Interface(requete):
    result = Similarite_Requete_Document(requete)  
    return result

def Telecharger_Json():
    global chemin_json
    if chemin_json:
        return gr.File(value=chemin_json) 
    return None  

#l'onglet  pour charger des documents 
interface_charger = gr.Interface(
    fn=Charger_Fichier_Interface,
    inputs=gr.File(label="Charger le fichier texte"),
    outputs="text",
    title="Charger et Sauvegarder des Documents",
    description="Cliquez sur le bouton pour charger et sauvegarder les documents avec leurs embeddings sous forme de fichier JSON."
)

#l'onglet pour la recherche 
interface_recherche = gr.Interface(
    fn=Recherche_Similarite_Interface,
    inputs=gr.Textbox(label="Entrez votre requête", placeholder="Saisir une requête ici...", lines=2),
    outputs="text",
    title="Recherche d'Articles",
    description="Entrez votre requête et obtenez les articles les plus pertinents."
)

#l'onglet pour le telechargement 
interface_telechargement = gr.Interface(
    fn=Telecharger_Json,
    inputs=None,  
    outputs=gr.File(label="Téléchargez le fichier JSON"),
    title="Télécharger le fichier JSON",
    description="Téléchargez le fichier JSON contenant les embeddings des documents."
)

#lancer l'interface
demo = gr.TabbedInterface([interface_recherche, interface_charger, interface_telechargement], ["Recherche", "Charger Document", "Télécharger JSON"])
demo.launch()

# hello

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
